In [23]:
from conllu import parse

with open('data/latin.conllu', 'r', encoding='utf8') as file:
    data = file.read()

sentences = parse(data)

import spacy
from spacy.tokens import Doc, Token
from conllu import parse_incr
from conllu import parse
from spacy.vocab import Vocab
from spacy import displacy
from pprint import pprint

nlp = spacy.blank("la")  
vocab = nlp.vocab

if not Token.has_extension("xpos"):
    Token.set_extension("xpos", default=None)
if not Token.has_extension("deps"):
    Token.set_extension("deps", default=None)
if not Token.has_extension("misc"):
    Token.set_extension("misc", default=None)


def process_conllu_file(file_path):
    docs = []
    with open(file_path, "r", encoding="utf-8") as file:
        for conllu_sentence in parse_incr(file):
            doc = conllu_to_spacy_doc(conllu_sentence, vocab)
            docs.append(doc)
    return docs

def conllu_to_spacy_doc(conllu_sentence, vocab):
    words = [token["form"] for token in conllu_sentence]
    heads = [token["head"] if token["head"] is not None else 0 for token in conllu_sentence]  # Head = 0 para raíces
    deps = [token["deprel"].split(":")[0] for token in conllu_sentence]
    lemmas = [token["lemma"] for token in conllu_sentence]
    pos_tags = [token["upostag"] for token in conllu_sentence]
    xpos_tags = [token["xpos"] for token in conllu_sentence]
    morph_feats = [token["feats"] or {} for token in conllu_sentence]
    misc_info = [token["misc"] or {} for token in conllu_sentence]
    deps_info = [token["deps"] or {} for token in conllu_sentence]

    # Crear el objeto Doc
    doc = Doc(vocab, words=words)

    # Asignar los atributos a cada token
    for i, token in enumerate(doc):
        token.lemma_ = lemmas[i]
        
        # Asignar el Universal POS a `pos_`, o usar `tag_` si no es válido
        if pos_tags[i] != "_":
            token.pos_ = pos_tags[i]
        elif xpos_tags[i] is not None:  # Verificar si xpos_tags[i] no es None antes de asignarlo
            token.tag_ = xpos_tags[i]  # Usar `xpos` como `tag_` si `upos` es "_"

        token.dep_ = deps[i]
        
        # Verificar que heads[i] sea un número entero antes de asignarlo como cabeza
        if isinstance(heads[i], int) and heads[i] > 0:
            token.head = doc[heads[i] - 1]  # Usar head como índice (spaCy usa índice 0 para la raíz)
        else:
            token.head = token  # Si head es 0 o no válido, el token se apunta a sí mismo como raíz

        # Asignar xpos, deps y misc como atributos personalizados
        token._.xpos = xpos_tags[i]
        token._.deps = deps_info[i]
        token._.misc = misc_info[i]

        # Convertir `feats` a un formato que spaCy entiende y asignarlo a morph
        if morph_feats[i]:
            feats_str = "|".join(f"{k}={v}" for k, v in morph_feats[i].items())
            token.set_morph(feats_str)

    return doc

def render_doc_by_sent_id(docs, sentences, target_sent_id):
    # Buscar el índice del `sent_id` deseado en los metadatos de `sentences`
    for i, sentence in enumerate(sentences):
        if sentence.metadata.get("sent_id") == target_sent_id:
            # Renderizar el Doc correspondiente usando `displacy`
            displacy.render(docs[i], style="dep", jupyter=True)
            return  # Salimos después de encontrar y renderizar
    print(f"Oración con sent_id '{target_sent_id}' no encontrada.")


file_path = "data/latin.conllu"
docs = process_conllu_file(file_path)


IN

In [24]:
render_doc_by_sent_id(docs, sentences, '1')
# The cat is in the wardrobe.

print(sentences[0])

for token in sentences[0]:
    print(token.items())

TokenList<Fēlēs, est, in, armāriō, ., metadata={generator: "UDPipe 2, https://lindat.mff.cuni.cz/services/udpipe", udpipe_model: "latin-ittb-ud-2.12-230717", udpipe_model_licence: "CC BY-NC-SA", newdoc: "None", newpar: "None", sent_id: "1", text: "Fēlēs est in armāriō."}>
dict_items([('id', 1), ('form', 'Fēlēs'), ('lemma', 'fēlēs'), ('upos', 'NOUN'), ('xpos', 'C1|grn1|casA|gen1'), ('feats', {'Case': 'Nom', 'Gender': 'Masc', 'InflClass': 'IndEurI', 'Number': 'Sing'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '0:5'})])
dict_items([('id', 2), ('form', 'est'), ('lemma', 'sum'), ('upos', 'AUX'), ('xpos', 'N3|modA|tem1|gen6'), ('feats', {'InflClass': 'LatAnom', 'Number': 'Sing', 'VerbForm': 'Fin'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '6:9'})])
dict_items([('id', 3), ('form', 'in'), ('lemma', 'in'), ('upos', 'ADP'), ('xpos', 'S4'), ('feats', None), ('head', 4), ('deprel', 'case'), ('deps', None), ('misc', {'TokenRange': '1

In [25]:
render_doc_by_sent_id(docs, sentences, '2')
# She enters the shop.


print(sentences[1])

for token in sentences[1]:
    print(token.items())

TokenList<Ea, intrat, tabernam, ., metadata={sent_id: "2", text: "Ea intrat tabernam."}>
dict_items([('id', 1), ('form', 'Ea'), ('lemma', 'ea'), ('upos', 'PRON'), ('xpos', 'F1|grn1|casA|gen2'), ('feats', {'Case': 'Abl', 'Gender': 'Fem', 'InflClass': 'LatPron', 'Number': 'Sing', 'Person': '3', 'PronType': 'Prs'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '22:24'})])
dict_items([('id', 2), ('form', 'intrat'), ('lemma', 'intro'), ('upos', 'VERB'), ('xpos', 'J3|modA|tem1|gen6'), ('feats', {'Aspect': 'Imp', 'InflClass': 'LatA', 'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Pres', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '25:31'})])
dict_items([('id', 3), ('form', 'tabernam'), ('lemma', 'tabernus'), ('upos', 'NOUN'), ('xpos', 'A1|grn1|casD|gen2'), ('feats', {'Case': 'Acc', 'Gender': 'Fem', 'InflClass': 'IndEurA', 'Number': 'Sing'}), ('head', 2), ('deprel', 'obj'), ('deps', None),

In [27]:
render_doc_by_sent_id(docs, sentences, '3')
# He came out of the cave.


print(sentences[2])

for token in sentences[2]:
    print(token.items())

TokenList<Ille, ēgreditur, ē, spēluncā, ., metadata={sent_id: "3", text: "Ille ēgreditur ē spēluncā."}>
dict_items([('id', 1), ('form', 'Ille'), ('lemma', 'ille'), ('upos', 'DET'), ('xpos', 'F1|grn1|casA|gen1'), ('feats', {'Case': 'Nom', 'Gender': 'Masc', 'InflClass': 'LatPron', 'Number': 'Sing', 'PronType': 'Dem'}), ('head', 2), ('deprel', 'nsubj:pass'), ('deps', None), ('misc', {'TokenRange': '42:46'})])
dict_items([('id', 2), ('form', 'ēgreditur'), ('lemma', 'ēgredo'), ('upos', 'VERB'), ('xpos', 'L3|modJ|tem1|gen6'), ('feats', {'Aspect': 'Imp', 'InflClass': 'LatX', 'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Pres', 'VerbForm': 'Fin', 'Voice': 'Pass'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '47:56'})])
dict_items([('id', 3), ('form', 'ē'), ('lemma', 'ē'), ('upos', 'ADP'), ('xpos', 'S4|vgr1'), ('feats', None), ('head', 4), ('deprel', 'case'), ('deps', None), ('misc', {'TokenRange': '57:58'})])
dict_items([('id', 4), ('form', 'spēluncā')

ON

In [28]:
render_doc_by_sent_id(docs, sentences, '4')
# The book is on the table.


print(sentences[3])

for token in sentences[3]:
    print(token.items())

TokenList<Liber, est, in, mēnsā, ., metadata={sent_id: "4", text: "Liber est in mēnsā."}>
dict_items([('id', 1), ('form', 'Liber'), ('lemma', 'liber'), ('upos', 'NOUN'), ('xpos', 'B1|grn1|casA|gen1'), ('feats', {'Case': 'Nom', 'Gender': 'Masc', 'InflClass': 'IndEurO', 'Number': 'Sing'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '69:74'})])
dict_items([('id', 2), ('form', 'est'), ('lemma', 'sum'), ('upos', 'AUX'), ('xpos', 'N3|modA|tem1|gen6'), ('feats', {'InflClass': 'LatAnom', 'Number': 'Sing', 'VerbForm': 'Fin'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '75:78'})])
dict_items([('id', 3), ('form', 'in'), ('lemma', 'in'), ('upos', 'ADP'), ('xpos', 'S4'), ('feats', None), ('head', 4), ('deprel', 'case'), ('deps', None), ('misc', {'TokenRange': '79:81'})])
dict_items([('id', 4), ('form', 'mēnsā'), ('lemma', 'mēnsā'), ('upos', 'NOUN'), ('xpos', 'C1|grn1|casF|gen2'), ('feats', {'Case': 'Abl', 'Gender': 'Fem', 'InflClass': 'I

In [29]:
render_doc_by_sent_id(docs, sentences, '5')
# The boy jumped onto the table.


print(sentences[4])

for token in sentences[4]:
    print(token.items())

TokenList<Puer, sālīvit, in, mēnsam, ., metadata={sent_id: "5", text: "Puer sālīvit in mēnsam."}>
dict_items([('id', 1), ('form', 'Puer'), ('lemma', 'puer'), ('upos', 'NOUN'), ('xpos', 'B1|grn1|casA|gen1'), ('feats', {'Case': 'Nom', 'Gender': 'Masc', 'InflClass': 'IndEurO', 'Number': 'Sing'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '89:93'})])
dict_items([('id', 2), ('form', 'sālīvit'), ('lemma', 'sālīvo'), ('upos', 'VERB'), ('xpos', 'L3|modA|tem1|gen6'), ('feats', {'Aspect': 'Imp', 'InflClass': 'LatX', 'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Pres', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '94:101'})])
dict_items([('id', 3), ('form', 'in'), ('lemma', 'in'), ('upos', 'ADP'), ('xpos', 'S4'), ('feats', None), ('head', 4), ('deprel', 'case'), ('deps', None), ('misc', {'TokenRange': '102:104'})])
dict_items([('id', 4), ('form', 'mēnsam'), ('lemma', 'mēnsa'), ('upos', 'NO

In [30]:
render_doc_by_sent_id(docs, sentences, '6')
# The girl climbed down from the table.


print(sentences[5])

for token in sentences[5]:
    print(token.items())

TokenList<Puella, dēscendit, dē, mēnsā, ., metadata={sent_id: "6", text: "Puella dēscendit dē mēnsā."}>
dict_items([('id', 1), ('form', 'Puella'), ('lemma', 'puella'), ('upos', 'NOUN'), ('xpos', 'A1|grn1|casA|gen2|vgr1'), ('feats', {'Case': 'Nom', 'Gender': 'Fem', 'InflClass': 'IndEurA', 'Number': 'Sing'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '113:119'})])
dict_items([('id', 2), ('form', 'dēscendit'), ('lemma', 'dēscendo'), ('upos', 'VERB'), ('xpos', 'L3|modA|tem1|gen6'), ('feats', {'Aspect': 'Imp', 'InflClass': 'LatX', 'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Pres', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '120:129'})])
dict_items([('id', 3), ('form', 'dē'), ('lemma', 'dē'), ('upos', 'ADP'), ('xpos', 'S4'), ('feats', None), ('head', 4), ('deprel', 'case'), ('deps', None), ('misc', {'TokenRange': '130:132'})])
dict_items([('id', 4), ('form', 'mēnsā'), ('lemma', 'm

AT

In [31]:
render_doc_by_sent_id(docs, sentences, '7')
# The bird rests next to the tree.


print(sentences[6])

for token in sentences[6]:
    print(token.items())


TokenList<Avis, requiēscit, prope, arborem, ., metadata={sent_id: "7", text: "Avis requiēscit prope arborem."}>
dict_items([('id', 1), ('form', 'Avis'), ('lemma', 'avis'), ('upos', 'PRON'), ('xpos', 'B1|grn1|casL|gen1'), ('feats', {'Case': 'Dat', 'Gender': 'Masc', 'InflClass': 'IndEurO', 'Number': 'Plur'}), ('head', 2), ('deprel', 'obl:arg'), ('deps', None), ('misc', {'TokenRange': '140:144'})])
dict_items([('id', 2), ('form', 'requiēscit'), ('lemma', 'requiēsco'), ('upos', 'VERB'), ('xpos', 'L3|modA|tem1|gen6'), ('feats', {'Aspect': 'Imp', 'InflClass': 'LatX', 'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Pres', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '145:155'})])
dict_items([('id', 3), ('form', 'prope'), ('lemma', 'prope'), ('upos', 'ADP'), ('xpos', 'S4'), ('feats', None), ('head', 4), ('deprel', 'case'), ('deps', None), ('misc', {'TokenRange': '156:161'})])
dict_items([('id', 4), ('form', 'arborem'), 

In [32]:
render_doc_by_sent_id(docs, sentences, '8')
# The boy runs toward the tree.



print(sentences[7])

for token in sentences[7]:
    print(token.items())

TokenList<Puer, currit, ad, arborem, ., metadata={sent_id: "8", text: "Puer currit ad arborem."}>
dict_items([('id', 1), ('form', 'Puer'), ('lemma', 'puer'), ('upos', 'NOUN'), ('xpos', 'B1|grn1|casA|gen1'), ('feats', {'Case': 'Nom', 'Gender': 'Masc', 'InflClass': 'IndEurO', 'Number': 'Sing'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '171:175'})])
dict_items([('id', 2), ('form', 'currit'), ('lemma', 'curro'), ('upos', 'VERB'), ('xpos', 'L3|modA|tem1|gen6'), ('feats', {'Aspect': 'Imp', 'InflClass': 'LatX', 'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Pres', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '176:182'})])
dict_items([('id', 3), ('form', 'ad'), ('lemma', 'ad'), ('upos', 'ADP'), ('xpos', 'S4'), ('feats', None), ('head', 4), ('deprel', 'case'), ('deps', None), ('misc', {'TokenRange': '183:185'})])
dict_items([('id', 4), ('form', 'arborem'), ('lemma', 'arbor'), ('upos', '

In [33]:
render_doc_by_sent_id(docs, sentences, '9')
# The squirrel moved away from the tree.



print(sentences[8])

for token in sentences[8]:
    print(token.items())

TokenList<Sciūrus, ab, arbore, sēcessit, ., metadata={sent_id: "9", text: "Sciūrus ab arbore sēcessit."}>
dict_items([('id', 1), ('form', 'Sciūrus'), ('lemma', 'sciūrus'), ('upos', 'NOUN'), ('xpos', 'B1|grn1|casA|gen1'), ('feats', {'Case': 'Nom', 'Gender': 'Masc', 'InflClass': 'IndEurO', 'Number': 'Sing'}), ('head', 4), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '195:202'})])
dict_items([('id', 2), ('form', 'ab'), ('lemma', 'ab'), ('upos', 'ADP'), ('xpos', 'S4|vgr2'), ('feats', None), ('head', 3), ('deprel', 'case'), ('deps', None), ('misc', {'TokenRange': '203:205'})])
dict_items([('id', 3), ('form', 'arbore'), ('lemma', 'arbor'), ('upos', 'NOUN'), ('xpos', 'C1|grn1|casF|gen2'), ('feats', {'Case': 'Abl', 'Gender': 'Fem', 'InflClass': 'IndEurX', 'Number': 'Sing'}), ('head', 4), ('deprel', 'obl:arg'), ('deps', None), ('misc', {'TokenRange': '206:212'})])
dict_items([('id', 4), ('form', 'sēcessit'), ('lemma', 'sēcedo'), ('upos', 'VERB'), ('xpos', 'L3|modA|tem4|gen6'), (